In [51]:
import pandas as pd
import glob
import os
from datetime import datetime
import calendar

anaconda not found


In [91]:
# Define the folder path (adjust as needed)
folder_path = './data/'  # e.g., './data/', or 'C:/my_folder/'
# Use glob to find all .csv files
csv_files = glob.glob(os.path.join(folder_path, "*.CSV"))

In [ ]:
# Define the folder path (adjust as needed)
folder_path = './interval/'  # e.g., './data/', or 'C:/my_folder/'
# Use glob to find all .csv files
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

In [92]:

# Check if any files found
if not csv_files:
    print("No CSV files found.")
else:
    # Read each CSV file and store in a list
    dfs = []
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            dfs.append(df)
            print(f"Loaded: {file}")
        except Exception as e:
            print(f"Error reading {file}: {e}")
    # Concatenate all DataFrames
    df = pd.concat(dfs, ignore_index=True)
    print(f"\nTotal rows after combining: {len(df)}")
df = df.drop(columns=["ESTIMATED_ACTUAL","REVISION_DATE", "CONSUMPTION_SURPLUSGENERATION"])
df['USAGE_START_TIME'] = df['USAGE_START_TIME'].str.strip()
df['USAGE_END_TIME'] = df['USAGE_END_TIME'].str.strip()
df['hour'] = pd.to_datetime(df['USAGE_START_TIME'], format='%H:%M').dt.hour

# Convert USAGE_DATE to datetime
df['datetime'] = pd.to_datetime(df['USAGE_DATE'], format='%m/%d/%Y')
# Create 15-min interval offsets (0, 15, 30, 45 minutes)
# Assuming rows are ordered chronologically
df['datetime'] += pd.to_timedelta(df['hour'], unit='hour')
# Now you have full timestamps
# print(df[['USAGE_DATE', 'USAGE_KWH', 'datetime']])


Loaded: ./data/IntervalMeterUsage202512232103511433F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512132104342333F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512092104337143F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512062104465213F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512262104295393F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512292104025003F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512242103557593F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512112104178453F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512252104042683F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512072104407293F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./data/IntervalMeterUsage202512032104445033F4F9339E4024C89B21685CA1223B8AEEML.CSV
Loaded: ./

In [93]:
# df

In [94]:
# Set datetime as index for resampling
df.set_index('datetime', inplace=True)
# Resample and sum kWh
hourly = df['USAGE_KWH'].resample('h').sum()
daily = df['USAGE_KWH'].resample('D').sum()
weekly = df['USAGE_KWH'].resample('W').sum()
monthly = df['USAGE_KWH'].resample('ME').sum()
# date =df['USAGE_DATE']

In [95]:
summary_df = pd.DataFrame({
    'hourly_kwh': hourly,
    'daily_kwh': daily,
    'weekly_kwh': weekly,
    'monthly_kwh': monthly
}).reset_index()
# Rename index to 'datetime'
summary_df.rename(columns={'index': 'datetime'}, inplace=True)

# Extract frequency labels from datetime
df.reset_index(inplace=True)
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.date
df['week'] = df['datetime'].dt.to_period('W')
df['month'] = df['datetime'].dt.to_period('M')
# Group by hour/day/week/month and aggregate
hourly_agg = df.groupby('hour')['USAGE_KWH'].sum().rename('hourly_kwh')
daily_agg = df.groupby('day')['USAGE_KWH'].sum().rename('daily_kwh')
weekly_agg = df.groupby('week')['USAGE_KWH'].sum().rename('weekly_kwh')
monthly_agg = df.groupby('month')['USAGE_KWH'].sum().rename('monthly_kwh')
# Merge back using map
df['hourly_kwh'] = df['hour'].map(hourly_agg)
df['daily_kwh'] = df['day'].map(daily_agg)
df['weekly_kwh'] = df['week'].map(weekly_agg)
df['monthly_kwh'] = df['month'].map(monthly_agg)

In [96]:
def is_month_end(date_input):
    # accepts date_input as datetime.date, datetime.datetime, or "YYYY-MM-DD" str
    if isinstance(date_input, str):
        date = datetime.strptime(date_input, "%Y-%m-%d").date()
    elif isinstance(date_input, datetime):
        date = date_input.date()
    else:
        date = date_input  # assume datetime.date
    last_day = calendar.monthrange(date.year, date.month)[1]
    return date.day == last_day

In [97]:
# summary_df = pd.DataFrame({
#     'hourly_kwh': [0.51, 0.48, 0.62],
#     'daily_kwh': [18.7, 19.2, 17.9],
#     'weekly_kwh': [125.4, 130.1, 122.8],
#     'monthly_kwh': [567.2, 571.0, 563.5]
# })
# Set datetime as index (if not already)
# summary_df.index = pd.to_datetime(datetime)
# summary_df.rename_axis('datetime', inplace=True)

                       # Reset index to make datetime a column
# summary_df = summary_df.reset_index()
# Extract date and time
summary_df['time'] = summary_df['datetime'].dt.strftime('%H:%M')
summary_df['date'] = summary_df['datetime'].dt.strftime('%Y-%m-%d')


In [98]:
# summary_df

In [99]:
# print(df.sort_values('datetime', ascending=False).head(10))

In [100]:
rythm_peak = ['18:00','19:00','20:00','21:00']


In [101]:
df = summary_df
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.day
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year
df['is_month_end']= df['datetime'].dt.is_month_end

In [102]:
# df[df['is_month_end'] == True][df['monthly_kwh'] >0]

In [103]:
def process(summary_df, peaks):
    df = summary_df
    df['hour'] = df['datetime'].dt.hour
    df['day'] = df['datetime'].dt.day
    df['month'] = df['datetime'].dt.month
    df['year'] = df['datetime'].dt.year
    df['is_month_end']= df['datetime'].dt.is_month_end
    
    # Ensure date and time are properly parsed
    # df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    # Extract hour from time
    # df['hour'] = df['datetime'].dt.hour
    # Define peak hours
    
    # Create peak flag
    df['is_peak'] = df['hour'].isin(peak_hours)
    # Separate peak and non-peak usage
    df['peak_kwh'] = df['hourly_kwh'].where(df['is_peak'], 0)
    df['non_peak_kwh'] = df['hourly_kwh'].where(~df['is_peak'], 0)
    # Set datetime as index for resampling
    df.set_index('datetime', inplace=True)
    # Daily sums
    df_daily = df.resample('D').agg({
        'peak_kwh': 'sum',
        'non_peak_kwh': 'sum'
    }).rename(columns={'peak_kwh': 'daily_peak_kwh', 'non_peak_kwh': 'daily_non_peak_kwh'})
    # Weekly sums
    df_weekly = df.resample('W-MON').agg({
        'peak_kwh': 'sum',
        'non_peak_kwh': 'sum'
    }).rename(columns={'peak_kwh': 'weekly_peak_kwh', 'non_peak_kwh': 'weekly_non_peak_kwh'})
    # Monthly sums
    df_monthly = df.resample('ME').agg({
        'peak_kwh': 'sum',
        'non_peak_kwh': 'sum'
    }).rename(columns={'peak_kwh': 'monthly_peak_kwh', 'non_peak_kwh': 'monthly_non_peak_kwh'})
    # Merge back into original DataFrame using the index
    df.reset_index(inplace=True)
    df = pd.merge(df, df_daily[['daily_peak_kwh', 'daily_non_peak_kwh']], 
                  left_on='datetime', right_index=True, how='left')
    df = pd.merge(df, df_weekly[['weekly_peak_kwh', 'weekly_non_peak_kwh']], 
                  left_on='datetime', right_index=True, how='left')
    df = pd.merge(df, df_monthly[['monthly_peak_kwh', 'monthly_non_peak_kwh']], 
                  left_on='datetime', right_index=True, how='left')
    # Fill NaNs with 0 (for first few days/weeks/months with no data)
    df.fillna(0, inplace=True)
    # Optional: Drop temporary columns if needed
    # df.drop(columns=['hour', 'is_peak', 'datetime'], inplace=True)
    return df

In [104]:
peak_hours = [18, 19, 20, 21]  # Convert to integers for comparison
rythm = process(summary_df, peak_hours)
rythm_monthly = rythm[rythm['is_month_end'] == True][rythm['monthly_kwh'] >0]
rythm_monthly = rythm_monthly[["datetime","monthly_kwh", "date", "day", "monthly_peak_kwh", "monthly_non_peak_kwh"]]
rythm_monthly.to_csv('output/rythm_monthly.csv', index=False)
rythm_monthly

/var/folders/8g/jn8cz2bj1hg4ydc3f57xd6m40000gn/T/ipykernel_61544/1174549103.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rythm_monthly = rythm[rythm['is_month_end'] == True][rythm['monthly_kwh'] >0]


,datetime,monthly_kwh,date,day,monthly_peak_kwh,monthly_non_peak_kwh
0,2025-11-30,57.087,2025-11-30,30,8.643,48.444
744,2025-12-31,1881.841,2025-12-31,31,293.003,1588.838


In [105]:
peak_hours = [18, 19, 20, 21]  # Convert to integers for comparison
rythm = process(summary_df, peak_hours)
rythm_daily = rythm[rythm['daily_kwh'] >0]
rythm_daily = rythm_daily[["datetime","daily_kwh", "date", "day", "daily_peak_kwh", "daily_non_peak_kwh"]]
rythm_daily.to_csv('output/rythm_daily.csv', index=False)
rythm_daily

,datetime,daily_kwh,date,day,daily_peak_kwh,daily_non_peak_kwh
0,2025-11-30,57.087,2025-11-30,30,8.643,48.444
24,2025-12-01,66.374,2025-12-01,1,15.065,51.309
48,2025-12-02,67.744,2025-12-02,2,11.883,55.861
72,2025-12-03,68.104,2025-12-03,3,11.758,56.346
96,2025-12-04,71.316,2025-12-04,4,13.202,58.114
120,2025-12-05,68.393,2025-12-05,5,13.522,54.871
144,2025-12-06,72.689,2025-12-06,6,7.898,64.791
168,2025-12-07,52.845,2025-12-07,7,7.733,45.112
192,2025-12-08,65.278,2025-12-08,8,12.675,52.603
216,2025-12-09,62.321,2025-12-09,9,9.633,52.688


In [106]:
peak_hours = [7, 8, 9, 10,11,12,13,14,15,16,17,18,19,20] 
non_peak = [21,22,23,0,1,2,3,4,5,6]
night_free_monthly = process(summary_df, peak_hours)
night_free_monthly = night_free_monthly[night_free_monthly['is_month_end'] == True][night_free_monthly['monthly_kwh'] >0]
night_free_monthly = night_free_monthly[["datetime","monthly_kwh", "date", "day", "monthly_peak_kwh", "monthly_non_peak_kwh"]]
night_free_monthly.to_csv('output/night_free_monthly.csv', index=False)
night_free_monthly

/var/folders/8g/jn8cz2bj1hg4ydc3f57xd6m40000gn/T/ipykernel_61544/1404146127.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  night_free_monthly = night_free_monthly[night_free_monthly['is_month_end'] == True][night_free_monthly['monthly_kwh'] >0]


,datetime,monthly_kwh,date,day,monthly_peak_kwh,monthly_non_peak_kwh
0,2025-11-30,57.087,2025-11-30,30,43.091,13.996
744,2025-12-31,1881.841,2025-12-31,31,1223.304,658.537


In [107]:
peak_hours = [7, 8, 9, 10,11,12,13,14,15,16,17,18,19,20] 
non_peak = [21,22,23,0,1,2,3,4,5,6]
night_free_daily = process(summary_df, peak_hours)
night_free_daily = night_free_daily[night_free_daily['daily_kwh'] >0]
night_free_daily = night_free_daily[["datetime","daily_kwh", "date", "day", "daily_peak_kwh", "daily_non_peak_kwh"]]
night_free_daily.to_csv('output/night_free_daily.csv', index=False)
night_free_daily

,datetime,daily_kwh,date,day,daily_peak_kwh,daily_non_peak_kwh
0,2025-11-30,57.087,2025-11-30,30,43.091,13.996
24,2025-12-01,66.374,2025-12-01,1,45.941,20.433
48,2025-12-02,67.744,2025-12-02,2,45.417,22.327
72,2025-12-03,68.104,2025-12-03,3,38.171,29.933
96,2025-12-04,71.316,2025-12-04,4,40.637,30.679
120,2025-12-05,68.393,2025-12-05,5,39.195,29.198
144,2025-12-06,72.689,2025-12-06,6,53.791,18.898
168,2025-12-07,52.845,2025-12-07,7,35.961,16.884
192,2025-12-08,65.278,2025-12-08,8,35.152,30.126
216,2025-12-09,62.321,2025-12-09,9,33.460,28.861


In [108]:
peak_hours = [6,7, 8, 9, 10,11,12,13,14,15,16,17,18,19,20,21,22] 
non_peak = [23,0,1,2,3,4,5]
chariot_night_free_daily = process(summary_df, peak_hours)
chariot_night_free_daily = chariot_night_free_daily[chariot_night_free_daily['daily_kwh'] >0]
chariot_night_free_daily = chariot_night_free_daily[["datetime","daily_kwh", "date", "day", "daily_peak_kwh", "daily_non_peak_kwh"]]
chariot_night_free_daily.to_csv('output/chariot_night_free_daily.csv', index=False)
chariot_night_free_daily

,datetime,daily_kwh,date,day,daily_peak_kwh,daily_non_peak_kwh
0,2025-11-30,57.087,2025-11-30,30,48.063,9.024
24,2025-12-01,66.374,2025-12-01,1,54.430,11.944
48,2025-12-02,67.744,2025-12-02,2,54.148,13.596
72,2025-12-03,68.104,2025-12-03,3,49.819,18.285
96,2025-12-04,71.316,2025-12-04,4,50.023,21.293
120,2025-12-05,68.393,2025-12-05,5,48.050,20.343
144,2025-12-06,72.689,2025-12-06,6,59.605,13.084
168,2025-12-07,52.845,2025-12-07,7,40.765,12.080
192,2025-12-08,65.278,2025-12-08,8,45.536,19.742
216,2025-12-09,62.321,2025-12-09,9,43.216,19.105


In [68]:
# rythm

In [69]:
# rythm.drop(columns=["hourly_kwh","daily_kwh", "time", "hour", "is_peak", "peak_kwh", "non_peak_kwh", "daily_peak_kwh", "daily_non_peak_kwh", weekly_peak_kwh", "weekly_non_peak_kwh"])

In [70]:
# rythm = rythm[["datetime","monthly_kwh", "date", "day", "monthly_peak_kwh", "monthly_non_peak_kwh"]]

In [71]:
# rythm

In [72]:
# rythm.to_csv('rythm_electricity_usage_with_peaks.csv', index=False)


In [73]:
# df['hour'] = pd.to_datetime(df['USAGE_START_TIME'], format=' %H:%M').dt.hour


In [74]:
# df

In [75]:
# df_monthly